# Elements Playground

Use this notebook to inspect Elements graphs, pull definitions from the FastAPI `/api/elements` endpoints, and simulate a run locally before wiring the graph to Control Center.

In [ ]:
GRAPH_PRESET = globals().get('GRAPH_PRESET', 'qa_loop')
GRAPH_ID = globals().get('GRAPH_ID', None)
ELEMENTS_API_URL = globals().get('ELEMENTS_API_URL', 'http://localhost:8000/api/elements')
OUTPUT_DIR = globals().get('OUTPUT_DIR', './_papermill')

In [ ]:
from pathlib import Path
import json
import sys
from urllib import request, error


def _discover_repo_root(start: Path) -> Path:
    for candidate in [start, *start.parents]:
        if (candidate / 'PROJECT_OVERVIEW.md').exists():
            return candidate
    return start


REPO_ROOT = _discover_repo_root(Path.cwd().resolve())
if str(REPO_ROOT) not in sys.path:
    sys.path.insert(0, str(REPO_ROOT))

from datalab.scripts import elements as elements_utils

OUTPUT_DIR = Path(OUTPUT_DIR)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
available_presets = elements_utils.available_presets()
available_presets

In [ ]:
def _fetch_graph(graph_id: str | None):
    if not graph_id:
        return elements_utils.load_graph(preset=GRAPH_PRESET)
    url = f"{ELEMENTS_API_URL.rstrip('/')}/graphs/{graph_id}"
    with request.urlopen(url, timeout=3.0) as response:
        return json.loads(response.read().decode('utf-8'))

try:
    graph_payload = _fetch_graph(GRAPH_ID)
except (error.URLError, OSError) as exc:
    graph_payload = elements_utils.load_graph(preset=GRAPH_PRESET)
    graph_payload.setdefault('metadata', {})['apiFallback'] = str(exc)

if 'tenantId' not in graph_payload and graph_payload.get('tenant_id'):
    graph_payload['tenantId'] = graph_payload.pop('tenant_id')
if 'workspaceId' not in graph_payload and graph_payload.get('workspace_id'):
    graph_payload['workspaceId'] = graph_payload.pop('workspace_id')
for edge in graph_payload.get('edges', []):
    if 'from' not in edge and edge.get('source'):
        edge['from'] = edge['source']
    if 'to' not in edge and edge.get('target'):
        edge['to'] = edge['target']

graph_payload

In [ ]:
graph_summary = elements_utils.graph_summary(graph_payload)
execution_plan = elements_utils.build_execution_plan(graph_payload)
graph_summary, execution_plan

In [ ]:
run_result = elements_utils.simulate_graph(graph_payload)
graph_slug = graph_summary['name'].lower().replace(' ', '_').replace('/', '-')
graph_path = elements_utils.write_graph(graph_payload, OUTPUT_DIR / f"{graph_slug}.json")
trace_path = elements_utils.export_trace(run_result['trace'], OUTPUT_DIR / f"{graph_slug}-trace.json")
run_result['artifacts'] = {'graph': str(graph_path), 'trace': str(trace_path)}
run_result